# Using regular expression with `re` module

Regular expression are tiny, highly specialized programming language
embedded inside Python. They are also called REs, regexes or regex 
patterns.

Using regexes you specify the rules for the set of possible strings 
that you want to match. You can then ask : 'Does this string match the pattern?' You can also use REs to modify a string or to split it apart it various ways.

Regexes are compiled into a series of bytecodes which are then executed by a matching engine written in C. 

Regexes language is relatively small and restricted. So, not all possible string processing tasks can be done using them. There are also tasks that can be done with regexes, but the resulting expression turn out to be very complicated. In this case you may be better off writing some Python code to process the text. Although Python code will be slower than regexes, it will be more understandable.


## Simple Patterns



We'll begin with the most common task: matching characters.


## Matching Characters

Most letters and characters will simply match themselves. There are exceptions for this rule: some characters are special metacharacters. They don't match themselves. Instead, they signal that some out-of-the-ordinary thing should be matched, or they affect other portions of the regex by repeating them or changing their meaning.

Here's a complete list of the metacharacters:

`. ^ $ * + ? { } [ ] \ | ( )` 

`[ ]` are used to specify a character class. It is a set of characters that you with to match. Characters can  be listed individually. To indicate a range of characters, we use two chars and separate them by `-`

F.e. `[abc]` will match any of the characters `a`, `b` or `c`. 
This is the same as `[a-c]`. 

If you want to match only lowercase letters, your regex will be : `[a-z]`

Metacharacters are not active inside classes. F.e. `[akm$]` will match any of the `a`, `k`, `m` or `$`. `$` is usually a metacharacter but inside a character class it's stripped of its special nature.

To match the characters not listed in the class, you should _complement_ the set. To do it, include `^` as the first character of the class. `^` outside a character class will simply match the `^` character. F.e., `[^5]` will match any character except `5`

Perhaps the most important metacharacter is the backslash `\`. It can be followed by various characters to signal various special sequences. It's also used to escape all  the metacharacters. So you can still match them in patterns. F.e. if you need to match a `[` or `\`, you can precede them with a backslash : `\[` or `\\` - it will remove their special meaning.

